In [13]:
import pandas as pd
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.schema import HumanMessage, SystemMessage
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

In [2]:
model_name = "ai-sage/GigaChat-20B-A3B-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(model=model_name, trust_remote_code=True, tensor_parallel_size=2, max_model_len=24000)
sampling_params = SamplingParams(temperature=0.3, max_tokens=1000)

INFO 12-17 16:02:04 config.py:1861] Downcasting torch.float32 to torch.float16.
INFO 12-17 16:02:07 config.py:1020] Defaulting to use mp for distributed inference
INFO 12-17 16:02:07 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='ai-sage/GigaChat-20B-A3B-instruct', speculative_config=None, tokenizer='ai-sage/GigaChat-20B-A3B-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=24000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), 

Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]


INFO 12-17 16:09:09 model_runner.py:1077] Loading model weights took 19.2353 GB
(VllmWorkerProcess pid=1920502) INFO 12-17 16:09:09 model_runner.py:1077] Loading model weights took 19.2353 GB
WARNING 12-17 16:09:10 fused_moe.py:324] Using default MoE config. Performance might be sub-optimal! Config file not found at /home/stepan2/miniconda3/envs/llama-env/lib/python3.10/site-packages/vllm/model_executor/layers/fused_moe/configs/E=64,N=896,device_name=NVIDIA_RTX_A6000.json
(VllmWorkerProcess pid=1920502) WARNING 12-17 16:09:10 fused_moe.py:324] Using default MoE config. Performance might be sub-optimal! Config file not found at /home/stepan2/miniconda3/envs/llama-env/lib/python3.10/site-packages/vllm/model_executor/layers/fused_moe/configs/E=64,N=896,device_name=NVIDIA_RTX_A6000.json
(VllmWorkerProcess pid=1920502) INFO 12-17 16:09:15 worker.py:232] Memory profiling results: total_gpu_memory=47.54GiB initial_memory_usage=19.86GiB peak_torch_memory=20.89GiB memory_usage_post_profile=19.9

In [8]:
file_path = 'house_dataset.csv'
loader = CSVLoader(file_path=file_path, encoding="utf-8")
docs = loader.load_and_split()

In [9]:
class SentenceTransformerEmbeddings:
    def __init__(self, model_name="intfloat/multilingual-e5-large-instruct"):
        self.model = SentenceTransformer(model_name)

    def embed_query(self, query):
        return self.model.encode([query], convert_to_tensor=False)[0]

    def embed_documents(self, docs):
        return self.model.encode(docs, convert_to_tensor=False)

embeddings = SentenceTransformerEmbeddings()

In [10]:
embedding_dim = len(embeddings.embed_query(" "))  
index = faiss.IndexFlatL2(embedding_dim)
vector_store = FAISS(
    embedding_function=embeddings.embed_query,  
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

texts = [doc.page_content for doc in docs]
vector_store.add_texts(texts)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


['939a7784-93af-4622-9cf3-8a260b4f64bb',
 'c6d04531-d531-4267-ba22-04375a74c76d',
 'eb9d9f8b-6e4e-4d41-95e6-7dfb9a614572',
 '8265b426-0419-4262-93d3-d5dc3d55ccc7',
 'cb9fd0f7-866c-4d91-ba3b-5fd8826a1fcd',
 '993c87b5-3e84-4d16-9f09-c764b0ee2b00',
 'a8607da7-4ef2-47c9-a794-5122b5f61c2d',
 'fa5d86ad-b99d-4237-814a-8e6a617c92ac',
 '5f7850e2-32ea-48b7-8741-c4620b33dfaa',
 'afa77ec4-0d81-4833-916a-971870ce020c',
 '32a931f8-3993-4b72-9b90-63c83a9865c8',
 '62dcd007-ac5a-40e2-89b4-9cb401d4fd0b',
 'fb6acb11-e23a-47af-aad0-21e32cfc7523',
 '2b58c02a-e98a-4aff-a549-f9e4ab1036ce',
 '1c7f841a-444b-4003-ac28-8a79ea9b8b82',
 'b797236c-d985-411d-b197-cd90a9536a89',
 'b570b102-e3b9-44c3-8aff-688b57235ba1',
 '44a1b29f-f751-4345-9c44-ba44c8ad22cf',
 'a07d2377-e018-48e1-8c94-bc94bb6f292d',
 '9446e35b-fffb-4110-af28-bb00ac591552',
 '5d9f538c-a866-4a9b-8551-596bf01aae2b',
 'bb7574a8-eae1-4163-b0a4-d89c0aa90744',
 'b9c6865f-44d2-4b49-8a75-546d7d1ebdc3',
 'f86fbdfd-8e9e-4f34-bcf3-bad9fe02da8f',
 '7ad281a3-4ef1-

In [11]:
def create_vllm_chain(retriever, tokenizer, llm, sampling_params, system_message):
    def answer_question(input_text):
        
        retrieved_docs = retriever.get_relevant_documents(input_text, k=10)
        context = "\n\n".join([doc.page_content for doc in retrieved_docs])
        
       
        messages = [
            {"role": "system", "content": system_message.content},
            {"role": "user", "content": f"Контекст: {context}\n\nВопрос: {input_text}"}
        ]
        
        
        prompt_token_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
        
      
        outputs = llm.generate(prompt_token_ids=[prompt_token_ids], sampling_params=sampling_params)
        
        
        generated_text = outputs[0].outputs[0].text
        return generated_text
    
    return answer_question

In [14]:
system_message = SystemMessage(content="Ты бот-помощник. Отвечай только на русском языке! Предоставь все релевантные данные о доме! ОБЯЗАТЕЛЬНО в конце отправляй номер телефона для связи и ссылку на объявление! Если нет идеально подходящего варианта, то обязательно скинь даже не самый лучший вариант!")
retriever = vector_store.as_retriever()
vllm_chain = create_vllm_chain(retriever, tokenizer, llm, sampling_params, system_message)

In [15]:
question = "нужен дом для вечеринки"
answer = vllm_chain(question)
print("Ответ:", answer)

/tmp/ipykernel_1920169/2103615022.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(input_text, k=10)
Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.86s/it, est. speed input: 828.05 toks/s, output: 77.86 toks/s]

Ответ: Предлагаю рассмотреть вариант аренды дома в Марушкинском поселении, д. Крекшино, улица Московская, 3А. Это место идеально подходит для проведения вечеринок и мероприятий. На первом этаже есть просторный зал с Алисой и Смарт ТВ, где можно установить пилон. Также есть кухня с необходимой техникой и мебелью. На втором этаже расположены три спальни и санузел с ванной. В доме могут разместиться до 12 человек. 

Для связи с владельцем дома, пожалуйста, используйте номер телефона +74952600803. 

Также, если вам интересен другой вариант, пожалуйста, уточните ваши предпочтения, и я постараюсь подобрать подходящий дом.
